# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [ ]:
import geopandas as gpd
from glob import glob
import glob,os
import matplotlib.pyplot as plt
import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
from math import sin, cos, sqrt, atan2, radians
import re

In [9]:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

## Part 1: Data Preprocessing

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:

def calculate_distance(from_coord, to_coord):
    R = 6373.0

    lat1 = radians(from_coord[1])
    lon1 = radians(from_coord[0])
    lat2 = radians(to_coord[1])
    lon2 = radians(to_coord[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(
        lambda x: calculate_distance([x['pickup_latitude'],x['pickup_longitude']],
                                     [x['dropoff_latitude'],x['dropoff_longitude']]),
        axis =1
    )

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [7]:


pattern = r"yellow_tripdata_201([0-4]{1})|yellow_tripdata_2009|yellow_tripdata_2015-0([1-6]{1})"

def get_taxi_html():
    response = requests.get(TAXI_URL)
    html = response.content
    return html

def find_taxi_csv_urls():
    soup = bs4.BeautifulSoup(get_taxi_html(), "html.parser")
    link_list = [a['href'] for a in soup.find_all('a')]
    new_link = []
    for item in link_list:
        match = re.findall(pattern,item)
        if len(match)>0 :
            new_link.append(item)
    return new_link

In [ ]:


#create the dataframe that converting pickup ID to lat/lon
taxi_zones = gpd.read_file("/Users/yzh/Documents/GitHub/taxi-cab-project/taxi_zones.shp")
taxi_zones = taxi_zones.to_crs(4326)
taxi_zones['Lon'] = taxi_zones.centroid.x
taxi_zones['Lat'] = taxi_zones.centroid.y
taxi_zones.drop(columns = "geometry", axis = 1,inplace = True)
taxi_zones = taxi_zones[['LocationID', 'Lon', 'Lat']]
taxi_zones_PU = taxi_zones
taxi_zones_PU.columns = ['PULocationID','pickup_longitude','pickup_latitude']

In [ ]:
#create the dataframe that converting dropoff ID to lat/lon
taxi_zones2 = gpd.read_file("/Users/yzh/Documents/GitHub/taxi-cab-project/taxi_zones.shp")
taxi_zones2 = taxi_zones2.to_crs(4326)
taxi_zones2['Lon'] = taxi_zones2.centroid.x
taxi_zones2['Lat'] = taxi_zones2.centroid.y
taxi_zones2.drop(columns = "geometry", axis = 1,inplace = True)
taxi_zones2 = taxi_zones2[['LocationID', 'Lon', 'Lat']]
taxi_zones_DO = taxi_zones2
taxi_zones_DO.columns = ['DOLocationID','dropoff_longitude','dropoff_latitude']

In [ ]:
def lat_long(df):
    idx_start = df.index[(df['pickup_latitude'] < 40.560445) | (df['pickup_longitude'] < -74.242330)].tolist()
    idx_end   = df.index[(df['dropoff_latitude'] > 40.908524) | (df['dropoff_longitude'] > -73.717047)].tolist()

  # Remove duplicate elemets
    idx_final = set(idx_start + idx_end)

  # Drop elements
    df.drop(idx_final, inplace=True)
    df = df.reset_index(drop=True)
    return df

def get_and_clean_month_taxi_data(url):
    data = requests.get(url).content
    name = link.rsplit("/",1)[-1]
    with open(name, 'wb') as f:
        f.write(data)
    df = pd.read_parquet(name)
    df = df.sample(frac=0.0002, replace=True, random_state=1)
    
    if "pickup_datetime" in df:
        #normalize the column name
        df.rename(columns = {"tip_amount":"Tip_Amt"}, inplace = True)
        
        #select the columns we need
        df=df[['pickup_datetime','dropoff_datetime','fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude',
               'dropoff_latitude','passenger_count','Tip_Amt']]
        df = df[df['passenger_count'] != 0]

        #filter trips with no fare
        df = df[df['fare_amount'] != 0]

        #filter trips with no distance between pickup and dropoff points
        df = df[(df['pickup_longitude'] != df['dropoff_longitude']) | 
                (df['pickup_longitude'] != df['dropoff_longitude'])]

        #remove unnecessary columns
        df=lat_long(df)
            
        #convert pickup and dropoff times to right type
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
        return df
        
    #the second pattern
    if "Start_Lon" in df:
        df.rename(columns = {'Start_Lon':'pickup_longitude', 'Start_Lat':'pickup_latitude','End_Lon':'dropoff_longitude',
                            'End_Lat':'dropoff_latitude',"Trip_Pickup_DateTime":"pickup_datetime",
                            'Trip_Dropoff_DateTime':'dropoff_datetime','Passenger_Count':'passenger_count',
                            'Fare_Amt':'fare_amount'}, inplace = True)
        #keeping columns we want
        df=df[['pickup_datetime','dropoff_datetime','fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude',
                        'dropoff_latitude','passenger_count','Tip_Amt']]
           
        #filter trips with zero passenger_count
        df = df[df['passenger_count'] != 0]

        #filter trips with no fare
        df = df[df['fare_amount'] != 0]

        #filter trips with no distance between pickup and dropoff points
        df = df[(df['pickup_longitude'] != df['dropoff_longitude']) | 
                    (df['pickup_longitude'] != df['dropoff_longitude'])]

        #remove unnecessary columns
        df=lat_long(df)
            
        #convert pickup and dropoff times to right type
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
        return df

    #third pattern
    #first we convert locationid to the latitudes and longitudes
    if "PULocationID" in df:
        df = pd.merge(df, taxi_zones_PU, how='left', on=['PULocationID'])
        df = pd.merge(df, taxi_zones_DO, how='left', on=['DOLocationID'])           
        
        #normalize the column name
        df.rename(columns = {"tip_amount":"Tip_Amt","tpep_pickup_datetime":"pickup_datetime", 'tpep_dropoff_datetime':'dropoff_datetime'}, inplace = True)
        
        #keeping columns we want
        df=df[['pickup_datetime','dropoff_datetime','fare_amount','pickup_longitude','pickup_latitude','dropoff_longitude',
                        'dropoff_latitude','passenger_count','Tip_Amt']]
        #filter trips with zero passenger_count
        df = df[df['passenger_count'] != 0]

        #filter trips with no fare
        df = df[df['fare_amount'] != 0]

        #filter trips with no distance between pickup and dropoff points
        df = df[(df['pickup_longitude'] != df['dropoff_longitude']) | 
                (df['pickup_longitude'] != df['dropoff_longitude'])]

        #remove trips not in coordinate box by calling functions
        df=lat_long(df)
            
        #convert pickup and dropoff times to right type
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
        df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
        return df 

In [ ]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

### Processing Uber Data

We are calling the uber data and cleaning it (removing missing data/NA, dropping coordinates outside our box)

In [10]:
def load_and_clean_uber_data(csv_file):
    
    #load csv file
    df = pd.read_csv(csv_file)
    
    #filter longitude
    df = df[(df['pickup_longitude'] <= -73.717047) & (df['dropoff_longitude'] <= -73.717047) & 
            (df['pickup_longitude'] >= -74.242330) & (df['dropoff_longitude'] >= -74.242330)]
    
    #filter latitude
    df = df[(df['pickup_latitude'] <= 40.908524) & (df['dropoff_latitude'] <= 40.908524) & 
            (df['pickup_latitude'] >= 40.560445) & (df['dropoff_latitude'] >= 40.560445)]
    
    #filter trips with zero passenger_count
    df = df[df['passenger_count'] != 0]
    
    #filter trips with no fare
    df = df[df['fare_amount'] != 0]
    
    #filter trips with no distance between pickup and dropoff points
    df = df[(df['pickup_longitude'] != df['dropoff_longitude']) | 
            (df['pickup_longitude'] != df['dropoff_longitude'])]
    
    #remove unnecessary columns
    df = df.drop(['Unnamed: 0','key'], axis=1)

    #change pickup_datetime from object type to datetime time
    df['pickup_datetime'] = df['pickup_datetime'].str.split(' ', n=2,expand=True)[1]
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

    return df

In [ ]:
UBER_DATA = "Downloads/uber_rides_sample.csv"
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

We are calling the weather data and cleaning it (filling in for missing values, dropping unecessary columns)

In [11]:
def clean_month_weather_data_hourly(csv_file):
    #form the hourly weather data
    df1 = csv_file[['DATE','HourlyPrecipitation','HourlyWindSpeed']]
    
    #change DATE from object type to datetime type
    df1['DATE'] = pd.to_datetime(df1['DATE'])
    
    #drop na rows
    df1 = df1[df1[['HourlyPrecipitation','HourlyWindSpeed']].notnull().any(axis=1)]
    
    return df1 

In [ ]:
def clean_month_weather_data_daily(csv_file):
    #form the daily weather data
    df2 = csv_file[['DATE','HourlyWindSpeed','DailyAverageWindSpeed', 'DailyPrecipitation', 'DailySustainedWindSpeed']]
    
    #change DATE from object type to datetime type
    df2['DATE'] = pd.to_datetime(df2['DATE'])
    
    #Get missing DailyAverageWindSpeed on 2014-01-26 by calculating the mean HourlyWindSpeed on 2014-01-26
    missing_day_df = df2[(df2['DATE'] > '2014-01-26') & (df2['DATE'] < '2014-01-27')]
    mean = missing_day_df.HourlyWindSpeed.mean()
    
    #drop na rows
    df2 = df2[df2[['DailyPrecipitation', 'DailyAverageWindSpeed', 'DailySustainedWindSpeed']].notnull().any(axis=1)]
    df2 = df2[['DATE','DailyPrecipitation', 'DailyAverageWindSpeed', 'DailySustainedWindSpeed']]
    
    #Update the missing value
    df2["DailyAverageWindSpeed"].fillna(mean,inplace=True)
    
    return df2

In [ ]:

def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    path = r'Desktop/weather'
    file = glob.glob(os.path.join(path, "*.csv"))
    print(file)
    weather_csv_files = []
    for f in file:
        weather_csv_files.append(pd.read_csv(f))
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

we compile the individual files into bigger dataframes for each data type (taxi, uber, weather)

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_data, daily_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

We know clean the data and store it in SQL databases. We will create four tables (taxi data, uber data, hourly weather data and daily weather data) with the necessary columns

In [14]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
   id INTEGER PRIMARY KEY,
   date DATE,
   hourly_precipitation FLOAT,
   hourly_wind_speed FLOAT
);

"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
   id INTEGER PRIMARY KEY,
   date DATE,
   daily_precipitation FLOAT,
   daily_wind_speed FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
   id INTEGER PRIMARY KEY,
   fare_amount FLOAT,
   pickup_datetime DATE,
   dropoff_datatime DATE,
   pickup_longitude FLOAT,
   pickup_latitude FLOAT,
   dropoff_longitude FLOAT,
   dropoff_latitude FLOAT,
   passenger_count INTEGER,
   Tip_Amout FLOAT,
   distance FLOAT
);

"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
   id INTEGER PRIMARY KEY,
   fare_amount FLOAT,
   pickup_datetime DATE,
   pickup_longitude FLOAT,
   pickup_latitude FLOAT,
   dropoff_longitude FLOAT,
   dropoff_latitude FLOAT,
   passenger_count INTEGER,
   distance FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [16]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

We know add our dataframes to the tables we created

In [17]:
def write_dataframes_to_table(table_to_df_dict):
    for key, value in table_to_df_dict.items():
        value.to_sql(key, con=engine, if_exists = 'append')

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [18]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)

### Query N

We know create queries that will select the columns and data needed to answer questions and explore the data

In [ ]:
QUERY_1 = """
SELECT hour, COUNT(*) as num_trips
FROM (
  SELECT strftime('%H', pickup_datetime) as hour
  FROM taxi_trips
)
GROUP BY hour
ORDER BY num_trips DESC
"""

In [ ]:
QUERY_2 = """
SELECT wkday, COUNT(*) as num_trips
FROM (
  SELECT strftime('%w', pickup_datetime) as wkday
  FROM uber_trips
)
GROUP BY wkday
ORDER BY num_trips DESC
"""

In [ ]:
QUERY_3 =  """
SELECT distance FROM(
SELECT distance FROM taxi_trips
UNION
SELECT distance FROM uber_trips
)
LIMIT 1
OFFSET(
SELECT COUNT(*) FROM (
SELECT distance FROM taxi_trips
UNION
SELECT distance FROM uber_trips
)
) * 95/100 - 1
"""

In [ ]:
QUERY_4 = """ 
SELECT strftime('%Y-%m-%d', pickup_datetime), AVG(distance) as avg_distance
FROM (
  SELECT pickup_datetime, distance FROM taxi_trips 
  WHERE strftime('%Y', pickup_datetime) = '2009' 
  UNION ALL
  SELECT pickup_datetime, distance FROM uber_trips 
  WHERE strftime('%Y', pickup_datetime) = '2009'
)
GROUP BY strftime('%Y-%m-%d', pickup_datetime)
ORDER BY COUNT(*) DESC
LIMIT 10
"""

In [ ]:
QUERY_5 = """ 
SELECT strftime('%Y-%m-%d', pickup_datetime), COUNT(*) as num_trips
FROM (
  SELECT pickup_datetime FROM taxi_trips 
  WHERE strftime('%Y', pickup_datetime) = '2014' 
  UNION ALL
  SELECT pickup_datetime FROM uber_trips 
  WHERE strftime('%Y', pickup_datetime) = '2014'
)
WHERE strftime('%Y-%m-%d', pickup_datetime) IN (
  SELECT strftime('%Y-%m-%d', DATE) FROM daily_weather
  WHERE strftime('%Y', DATE) = '2014'
  ORDER BY DailyAverageWindSpeed DESC
)
GROUP BY strftime('%Y-%m-%d', pickup_datetime)
ORDER BY COUNT(*) DESC
LIMIT 10
"""

In [ ]:
QUERY_6 = """ 
SELECT strftime('%Y-%m-%d %H', pickup_datetime) as hourtime, COUNT(*) as num_trips
FROM (
  SELECT pickup_datetime FROM taxi_trips 
  WHERE taxi_trips.pickup_datetime BETWEEN '2012-10-22' AND '2012-11-05' 
  UNION ALL
  SELECT pickup_datetime FROM uber_trips 
  WHERE uber_trips.pickup_datetime BETWEEN '2012-10-22' AND '2012-11-05'
)
GROUP BY hourtime
"""

In [ ]:
engine.execute(QUERY_1).fetchall()
engine.execute(QUERY_2).fetchall()
engine.execute(QUERY_3).fetchall()
engine.execute(QUERY_4).fetchall()
engine.execute(QUERY_5).fetchall()
engine.execute(QUERY_6).fetchall()

In [ ]:
write_query_to_file(QUERY_1, "Most_popular_hour_yellow_taxi_200901_201506.sql")
write_query_to_file(QUERY_2, "Most_popular_weekday_uber_200901_201506.sql")
write_query_to_file(QUERY_3, "95_percentile_distance_all_201307.sql")
write_query_to_file(QUERY_4, "Top10_highest_hired_day_with_avg_distance_2009.sql")
write_query_to_file(QUERY_5, "Top10_windiest_day_with_hired_trips_count_2014.sql")
write_query_to_file(QUERY_6, "Hurricane_Sandy_data.sql")

### Visualization N

We know create different graphics that showcase the data. The data used for the graphs is called from our SQL tables. Simply just call each function to display the graph

In [19]:


def hours_taxi_trips():

    # Get the results from the query and store them in two lists
    hour, number_trips = [], []
    
    for row in engine.execute(QUERY_1 ).fetchall():
        hour.append(row[0])
        number_trips.append(row[1])

# Use matplotlib to create a bar chart showing the number of trips per hour
        plt.bar(hour, number_trips)
        plt.xlabel('Hour of the Day')
        plt.ylabel('Number of Taxi Trips')
        plt.title("Number of taxi trips taken for each hour of the day")
    plt.show()

In [ ]:
hours_taxi_trips()

In [ ]:
def avg_distance_month():
# get the average distance traveled per month for taxis and Ubers combined
    query_7 = """
        SELECT strftime('%m', pickup_datetime) as month, AVG(distance) as avg_distance
        FROM uber_trips,taxi_trips
        GROUP BY month
        ORDER BY month;
        """

    # Get the results from the query and store them in two lists
    months, avg_distances = [], []
    for row in engine.execute(query_7).fetchall():
        months.append(row[0])
        avg_distances.append(row[1])

    # Calculate the standard deviation and the 90% confidence interval for the average distance traveled per month
    stdev = np.std(avg_distances)
    confidence_interval = 1.645 * stdev
    upper_bound = [avg + confidence_interval for avg in avg_distances]
    lower_bound = [avg - confidence_interval for avg in avg_distances]

    # Use matplotlib to create a bar chart showing the average distance traveled per month,
    # with the 90% confidence interval around the mean
    plt.bar(months, avg_distances, yerr=[upper_bound, lower_bound])
    plt.xlabel('Month')
    plt.ylabel('Average Distance Traveled (km)')
    plt.title('Average distance traveled per month')
    plt.show()


avg_distance_month()

In [ ]:
def tip_distance():
    # retrieve the tip amount and distance for each Yellow Taxi ride
    query_08 = """SELECT tip_amount, distance FROM taxi_trips;"""
    results = engine.execute(query_08).fetchall()
    tip, distance = [], []
    for row in engine.execute(query_08).fetchall():
        tip.append(row[0])
        distance.append(row[1])


    # Create the scatter plot
    plt.scatter(tip, distance)
    plt.xlabel("Tip amount")
    plt.ylabel("Distance (km)")
    plt.title("Tip amount vs. Distance for Taxi rides")
    plt.show()


In [ ]:
tip_distance()

In [ ]:
def tip_rain():
    # retrieve the tip amount and distance for each Yellow Taxi ride
    query = """SELECT tip_amount, hourly_precipitation FROM taxi_trips,hourly_weather;"""
    results = engine.execute(query).fetchall()
    tip, precipitation = [], []
    for row in engine.execute(query).fetchall():
        tip.append(row[0])
        precipitation.append(row[1])


    # Create the scatter plot
    plt.scatter(tip, precipitation)
    plt.xlabel("Tip amount")
    plt.ylabel("Precipitation")
    plt.title("Tip amount vs. Precipitation for Taxi rides")
    plt.show()

In [ ]:
tip_rain()